# **Filtro de tipo de mercado DatAlpine**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "tulancingo" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo',
 'df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_oct_2022_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_sep_2023_tulancingo']

In [2]:
print(df_alfa_q_sep_2023_tulancingo.columns)
print(df_alfa_q_oct_2022_tulancingo.columns)
print(df_alfa_marzo_2024_tulancingo.columns)

Index(['id', 'q', 'categoria', 'ids', 'propiedad', 'precio', 'm_total',
       'm_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_publicacion', 'personas_interesadas',
       'promedio_interes_mensual', 'estado', 'tipo', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total', 'cantidad_pisos',
       'antiguedad', 'seguridad', 'fraccionamiento', 'colonia', 'cp', 'url',
       'latitud', 'longitud'],
      dtype='object')
Index(['id', 'q', 'categoria', 'ids', 'propiedad', 'precio', 'm_total',
       'm_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_publicacion', 'personas_interesadas',
       'promedio_interes_mensual', 'estado', 'tipo', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total', 'cantidad_pisos',
       'antiguedad', 'seguridad', 'fraccionamiento', 'colonia', 'cp', 'url',
       'latitud', 'longitud'],
      dtype='object')
Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total

In [3]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','m_total':'metros_total','m_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [4]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [5]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [6]:
df_total.columns

Index(['id', 'categoria', 'propiedad', 'precio', 'metros_total',
       'metros_construido', 'precio_m2_construido', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total'],
      dtype='object')

In [7]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1888 entries, 0 to 1887
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1888 non-null   int64  
 1   categoria             1888 non-null   object 
 2   propiedad             1888 non-null   object 
 3   precio                1888 non-null   int64  
 4   metros_total          1888 non-null   int64  
 5   metros_construido     1888 non-null   int64  
 6   precio_m2_construido  1888 non-null   float64
 7   estacionamiento       1888 non-null   int64  
 8   recamaras             1888 non-null   int64  
 9   baño                  1888 non-null   int64  
 10  medio_baño            1888 non-null   int64  
 11  baño_total            1888 non-null   float64
dtypes: float64(2), int64(8), object(2)
memory usage: 177.1+ KB


In [8]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

df_total = df_total.drop(columns=['categoria'])
df_total['categoria'] = df_total['precio'].apply(asignar_categoria)

In [9]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=8) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

# 1. Casa Promedio

In [10]:
df_total['segmento'] = df_total['categoria'].str[0]
# Calcular el promedio de estacionamientos, recamaras y baños por segmento
grupo_promedios = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    #return f"{valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles
    return f"{valor}"

# Aplicar la función a cada columna que necesitas formatear
#for i in range(3):  # 0, 1, 2, 3 corresponden a 'estacionamiento', 'recamaras', 'baño_total'
#    grupo_promedios.iloc[:, i] = grupo_promedios.iloc[:, i].apply(formato_monetario)
print(grupo_promedios)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    3.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      1.0        3.0    2.0
L                      4.0        5.0    5.0
S                      2.0        4.0    3.0


In [11]:
import plotly.graph_objects as go
import os
import plotly.io as pio
fig = go.Figure(data=[go.Table(
    header=dict(values=['Segmento','Estacionamientos', 'Recámaras', 'Baños'],
                align='left'),
    cells=dict(values=[grupo_promedios.index,  # Índices de categorías #'estacionamiento', 'recamaras', 'baño_total'
                       grupo_promedios['Estacionamientos'],  
                       grupo_promedios['Recámaras'], 
                       grupo_promedios['Baños']],  
               align='left'))
])
fig.show()

In [12]:
import plotly.io as pio
# Asegúrate de que la carpeta 'assets/tablas' exista
if not os.path.exists('assets/tablas'):
    os.makedirs('assets/tablas')

# Guardar cada fila en un archivo HTML separado
for index, row in grupo_promedios.iterrows():
    # Crear la tabla para cada fila
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Segmento','Estacionamientos', 'Recámaras', 'Baños'],
                    align='left'),
        cells=dict(values=[grupo_promedios.index,  # Índices de categorías #'estacionamiento', 'recamaras', 'baño_total'
                        grupo_promedios['Estacionamientos'],  
                        grupo_promedios['Recámaras'], 
                        grupo_promedios['Baños']],  
                align='left'))
    ])

    # Nombre del archivo HTML basado en el nombre del índice (segmento)
    nombre_archivo = f"assets/tablas/casa_media_{row.name}.html"
    # Guardar la tabla como archivo HTML
    pio.write_html(fig, nombre_archivo)

# 2. Casa Mediana

In [13]:
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=8) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

In [18]:
df_total['segmento'] = df_total['categoria'].str[0]
# Calcular el promedio de estacionamientos, recamaras y baños por segmento
grupo_promedios = df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].median()
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"{valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar la función a cada columna que necesitas formatear
#for i in range(3):  # 0, 1, 2, 3 corresponden a 'estacionamiento', 'recamaras', 'baño_total'
#    grupo_promedios.iloc[:, i] = grupo_promedios.iloc[:, i].apply(formato_monetario)
print(grupo_promedios)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    3.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      1.0        3.0    2.0
E                      1.0        3.0    1.0
L                      2.0        5.0    4.0
S                      2.0        4.0    3.0


In [19]:
import plotly.graph_objects as go
import os
import plotly.io as pio

# Crear la tabla con Plotly sin estilos personalizados
fig = go.Figure(data=[go.Table(
    header=dict(values=['Segmento','Estacionamientos', 'Recámaras', 'Baños'],
                align='left'),
    cells=dict(values=[grupo_promedios.index,  # Índices de categorías #'estacionamiento', 'recamaras', 'baño_total'
                       grupo_promedios['Estacionamientos'],  # Valores mínimos
                       grupo_promedios['Recámaras'], 
                       grupo_promedios['Baños']],  # Valores máximos
               align='left'))
])
fig.show()

In [20]:
import plotly.io as pio
# Asegúrate de que la carpeta 'assets/tablas' exista
if not os.path.exists('assets/tablas'):
    os.makedirs('assets/tablas')

# Guardar cada fila en un archivo HTML separado
for index, row in grupo_promedios.iterrows():
    # Crear la tabla para cada fila
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Segmento','Estacionamientos', 'Recámaras', 'Baños'],
                    align='left'),
        cells=dict(values=[grupo_promedios.index,  # Índices de categorías #'estacionamiento', 'recamaras', 'baño_total'
                        grupo_promedios['Estacionamientos'],  # Valores mínimos
                        grupo_promedios['Recámaras'], 
                        grupo_promedios['Baños']],  # Valores máximos
                align='left'))
    ])

    # Nombre del archivo HTML basado en el nombre del índice (segmento)
    nombre_archivo = f"assets/tablas/casa_mediana_{row.name}.html"
    # Guardar la tabla como archivo HTML
    pio.write_html(fig, nombre_archivo)

# 3. Precio Mercado

In [21]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['precio'].describe()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_descripcion[['min', 'max','mean','50%']]
grupo_filtrado.rename(columns={'50%': 'Mediana','min':'Mínimo','max':'Máximo','mean':'Promedio',}, inplace=True)
grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\2462652399.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Mínimo,Máximo,Promedio,Mediana
segmento,,,,
A,3300000.0,3946465.0,3.569255e+06,3555000.0
B,2586000.0,3249000.0,2.882170e+06,2890000.0
C,1773000.0,2499000.0,2.071011e+06,1995000.0
D,1010000.0,1737200.0,1.294082e+06,1266000.0
E,346040.0,995000.0,6.866023e+05,630000.0
L,12260000.0,17500000.0,1.605000e+07,17000000.0
S,4130000.0,11083000.0,5.722591e+06,4810000.0


In [22]:
# Suponiendo que ya tienes df_total y grupo_filtrado preparado como mencionaste
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['precio'].describe()
grupo_filtrado = grupo_descripcion[['min', 'max', 'mean', '50%']]
# Renombrar columnas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar la función a cada columna que necesitas formatear
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].apply(formato_monetario)
print(grupo_filtrado)

                  Mínimo          Máximo        Promedio         Mediana
segmento                                                                
A          $3,300,000.00   $3,946,465.00   $3,569,254.94   $3,555,000.00
B          $2,586,000.00   $3,249,000.00   $2,882,169.75   $2,890,000.00
C          $1,773,000.00   $2,499,000.00   $2,071,010.57   $1,995,000.00
D          $1,010,000.00   $1,737,200.00   $1,294,081.87   $1,266,000.00
E            $346,040.00     $995,000.00     $686,602.33     $630,000.00
L         $12,260,000.00  $17,500,000.00  $16,050,000.00  $17,000,000.00
S          $4,130,000.00  $11,083,000.00   $5,722,591.43   $4,810,000.00


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\3635529776.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\3635529776.py:19: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segmento
A     $3,300,000.00
B     $2,586,000.00
C     $1,773,000.00
D     $1,010,000.00
E       $346,040.00
L    $12,260,000.00
S     $4,130,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\3635529776.py:19: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segmento
A     $3,946,465.00
B     $3,249,000.00
C     $2,499

In [23]:
import plotly.graph_objects as go
import os
import plotly.io as pio

# Crear la tabla con Plotly sin estilos personalizados
fig = go.Figure(data=[go.Table(
    header=dict(values=['Segmento','Mínimo', 'Máximo', 'Promedio', 'Mediana'],
                align='left'),
    cells=dict(values=[grupo_filtrado.index,  # Índices de categorías
                       grupo_filtrado['Mínimo'],  # Valores mínimos
                       grupo_filtrado['Máximo'],
                       grupo_filtrado['Promedio'],
                       grupo_filtrado['Mediana']],  # Valores máximos
               align='left'))
])
fig.show()

In [24]:
import plotly.io as pio
# Asegúrate de que la carpeta 'assets/tablas' exista
if not os.path.exists('assets/tablas'):
    os.makedirs('assets/tablas')

# Guardar cada fila en un archivo HTML separado
for index, row in grupo_filtrado.iterrows():
    # Crear la tabla para cada fila
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Segmento','Mínimo', 'Máximo', 'Promedio', 'Mediana'],
                    align='left'),
        cells=dict(values=[grupo_filtrado.index,  # Índices de categorías
                        grupo_filtrado['Mínimo'],  # Valores mínimos
                        grupo_filtrado['Máximo'],
                        grupo_filtrado['Promedio'],
                        grupo_filtrado['Mediana']],  # Valores máximos
                align='left'))
    ])

    # Nombre del archivo HTML basado en el nombre del índice (segmento)
    nombre_archivo = f"assets/tablas/precio_mercado_{row.name}.html"
    # Guardar la tabla como archivo HTML
    pio.write_html(fig, nombre_archivo)

# 4. M2 Terreno

In [29]:
df_total['metros_total'].describe()

count    1888.000000
mean      169.227225
std       223.257465
min         0.000000
25%       100.000000
50%       120.000000
75%       180.000000
max      4904.000000
Name: metros_total, dtype: float64

In [30]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['metros_total'].describe()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_descripcion[['min', 'max','mean','50%']]
grupo_filtrado.rename(columns={'50%': 'Mediana','min':'Mínimo','max':'Máximo','mean':'Promedio',}, inplace=True)
grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\3995000513.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Mínimo,Máximo,Promedio,Mediana
segmento,,,,
A,107.0,1049.0,255.607595,200.0
B,70.0,265.0,173.915966,200.0
C,73.0,400.0,164.994048,146.0
D,67.0,404.0,136.974093,105.0
E,34.0,460.0,129.453901,110.0
L,220.0,1972.0,1024.888889,1000.0
S,0.0,4904.0,456.267857,228.0


In [31]:
# Suponiendo que ya tienes df_total y grupo_filtrado preparado como mencionaste
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['metros_total'].describe()
grupo_filtrado = grupo_descripcion[['min', 'max', 'mean', '50%']]
# Renombrar columnas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Función para formatear valores en formato monetario
def formato_monetario(valor):
    #return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles
    return f"{valor:,.2f} m²"

# Aplicar la función a cada columna que necesitas formatear
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].apply(formato_monetario)
print(grupo_filtrado)

             Mínimo       Máximo     Promedio      Mediana
segmento                                                  
A         107.00 m²  1,049.00 m²    255.61 m²    200.00 m²
B          70.00 m²    265.00 m²    173.92 m²    200.00 m²
C          73.00 m²    400.00 m²    164.99 m²    146.00 m²
D          67.00 m²    404.00 m²    136.97 m²    105.00 m²
E          34.00 m²    460.00 m²    129.45 m²    110.00 m²
L         220.00 m²  1,972.00 m²  1,024.89 m²  1,000.00 m²
S           0.00 m²  4,904.00 m²    456.27 m²    228.00 m²


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\2765143147.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\2765143147.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segmento
A    107.00 m²
B     70.00 m²
C     73.00 m²
D     67.00 m²
E     34.00 m²
L    220.00 m²
S      0.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\2765143147.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segmento
A    1,049.00 m²
B      265.00 m²
C      400.00 m²
D      404.00 m²
E      460.00 m²
L 

In [32]:
import plotly.graph_objects as go
import os
import plotly.io as pio

# Crear la tabla con Plotly sin estilos personalizados
fig = go.Figure(data=[go.Table(
    header=dict(values=['Segmento','Mínimo', 'Máximo', 'Promedio', 'Mediana'],
                align='left'),
    cells=dict(values=[grupo_filtrado.index,  # Índices de categorías
                       grupo_filtrado['Mínimo'],  # Valores mínimos
                       grupo_filtrado['Máximo'],
                       grupo_filtrado['Promedio'],
                       grupo_filtrado['Mediana']],  # Valores máximos
               align='left'))
])
fig.show()

In [33]:
import plotly.io as pio
# Asegúrate de que la carpeta 'assets/tablas' exista
if not os.path.exists('assets/tablas'):
    os.makedirs('assets/tablas')

# Guardar cada fila en un archivo HTML separado
for index, row in grupo_filtrado.iterrows():
    # Crear la tabla para cada fila
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Segmento', 'Mínimo', 'Máximo', 'Promedio', 'Mediana'],
                    align='left'),
        cells=dict(values=[[row.name],  # Segmento actual usando el índice
                           [row['Mínimo']],  # Valor mínimo actual
                           [row['Máximo']],  # Valor máximo actual
                           [row['Promedio']],  # Promedio actual
                           [row['Mediana']]],  # Mediana actual
                       align='left'))
    ])

    # Nombre del archivo HTML basado en el nombre del índice (segmento)
    nombre_archivo = f"assets/tablas/m2_terreno_{row.name}.html"
    # Guardar la tabla como archivo HTML
    pio.write_html(fig, nombre_archivo)

# 5. M2 Construcción

In [34]:
df_total['metros_construido'].describe()

count    1888.000000
mean      172.557203
std       108.517592
min         0.000000
25%        95.000000
50%       144.000000
75%       220.000000
max      1500.000000
Name: metros_construido, dtype: float64

In [35]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['metros_construido'].describe()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_descripcion[['min', 'max','mean','50%']]
grupo_filtrado.rename(columns={'50%': 'Mediana','min':'Mínimo','max':'Máximo','mean':'Promedio',}, inplace=True)
grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\3787532931.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Mínimo,Máximo,Promedio,Mediana
segmento,,,,
A,150.0,1049.0,277.974684,250.0
B,2.0,600.0,208.571429,159.0
C,73.0,404.0,186.541667,159.0
D,60.0,420.0,139.500000,120.0
E,1.0,420.0,147.445626,110.0
L,275.0,598.0,452.777778,500.0
S,0.0,1500.0,309.366071,300.0


In [36]:
# Suponiendo que ya tienes df_total y grupo_filtrado preparado como mencionaste
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['metros_construido'].describe()
grupo_filtrado = grupo_descripcion[['min', 'max', 'mean', '50%']]
# Renombrar columnas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Función para formatear valores en formato monetario
def formato_monetario(valor):
    #return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles
    return f"{valor:,.2f} m²"

# Aplicar la función a cada columna que necesitas formatear
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].apply(formato_monetario)
print(grupo_filtrado)

             Mínimo       Máximo   Promedio    Mediana
segmento                                              
A         150.00 m²  1,049.00 m²  277.97 m²  250.00 m²
B           2.00 m²    600.00 m²  208.57 m²  159.00 m²
C          73.00 m²    404.00 m²  186.54 m²  159.00 m²
D          60.00 m²    420.00 m²  139.50 m²  120.00 m²
E           1.00 m²    420.00 m²  147.45 m²  110.00 m²
L         275.00 m²    598.00 m²  452.78 m²  500.00 m²
S           0.00 m²  1,500.00 m²  309.37 m²  300.00 m²


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\1275041692.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\1275041692.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segmento
A    150.00 m²
B      2.00 m²
C     73.00 m²
D     60.00 m²
E      1.00 m²
L    275.00 m²
S      0.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\1275041692.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segmento
A    1,049.00 m²
B      600.00 m²
C      404.00 m²
D      420.00 m²
E      420.00 m²
L 

In [37]:
import plotly.graph_objects as go
import os
import plotly.io as pio

# Crear la tabla con Plotly sin estilos personalizados
fig = go.Figure(data=[go.Table(
    header=dict(values=['Segmento','Mínimo', 'Máximo', 'Promedio', 'Mediana'],
                align='left'),
    cells=dict(values=[grupo_filtrado.index,  # Índices de categorías
                       grupo_filtrado['Mínimo'],  # Valores mínimos
                       grupo_filtrado['Máximo'],
                       grupo_filtrado['Promedio'],
                       grupo_filtrado['Mediana']],  # Valores máximos
               align='left'))
])
fig.show()

In [38]:
import plotly.io as pio
# Asegúrate de que la carpeta 'assets/tablas' exista
if not os.path.exists('assets/tablas'):
    os.makedirs('assets/tablas')

# Guardar cada fila en un archivo HTML separado
for index, row in grupo_filtrado.iterrows():
    # Crear la tabla para cada fila
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Segmento', 'Mínimo', 'Máximo', 'Promedio', 'Mediana'],
                    align='left'),
        cells=dict(values=[[row.name],  # Segmento actual usando el índice
                           [row['Mínimo']],  # Valor mínimo actual
                           [row['Máximo']],  # Valor máximo actual
                           [row['Promedio']],  # Promedio actual
                           [row['Mediana']]],  # Mediana actual
                       align='left'))
    ])

    # Nombre del archivo HTML basado en el nombre del índice (segmento)
    nombre_archivo = f"assets/tablas/m2_construido_{row.name}.html"
    # Guardar la tabla como archivo HTML
    pio.write_html(fig, nombre_archivo)

# 6. Pecio M2 Construcción

In [39]:
df_total['precio_m2_construido'].describe()

count    1.888000e+03
mean     1.215868e+04
std      3.950819e+04
min      0.000000e+00
25%      5.508770e+03
50%      1.028571e+04
75%      1.375000e+04
max      1.390000e+06
Name: precio_m2_construido, dtype: float64

In [40]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['precio_m2_construido'].describe()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_descripcion[['min', 'max','mean','50%']]
grupo_filtrado.rename(columns={'50%': 'Mediana','min':'Mínimo','max':'Máximo','mean':'Promedio',}, inplace=True)
grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\2424566928.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Mínimo,Máximo,Promedio,Mediana
segmento,,,,
A,3145.85,26309.77,14548.690253,13750.000
B,5333.33,1390000.00,28011.628739,17666.670
C,4918.81,28273.97,12724.301964,11929.485
D,2607.14,27216.67,10934.142565,10991.665
E,1023.81,680000.00,8630.206052,5833.330
L,27591.97,45682.54,36897.535556,34000.000
S,0.00,49250.00,20666.750357,17916.670


In [41]:
# Suponiendo que ya tienes df_total y grupo_filtrado preparado como mencionaste
df_total['segmento'] = df_total['categoria'].str[0]
grupo_descripcion = df_total.groupby('segmento')['precio_m2_construido'].describe()
grupo_filtrado = grupo_descripcion[['min', 'max', 'mean', '50%']]
# Renombrar columnas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles
    #return f"{valor:,.2f}"

# Aplicar la función a cada columna que necesitas formatear
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].apply(formato_monetario)
print(grupo_filtrado)

              Mínimo         Máximo    Promedio     Mediana
segmento                                                   
A          $3,145.85     $26,309.77  $14,548.69  $13,750.00
B          $5,333.33  $1,390,000.00  $28,011.63  $17,666.67
C          $4,918.81     $28,273.97  $12,724.30  $11,929.49
D          $2,607.14     $27,216.67  $10,934.14  $10,991.67
E          $1,023.81    $680,000.00   $8,630.21   $5,833.33
L         $27,591.97     $45,682.54  $36,897.54  $34,000.00
S              $0.00     $49,250.00  $20,666.75  $17,916.67


C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\1363743977.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\1363743977.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segmento
A     $3,145.85
B     $5,333.33
C     $4,918.81
D     $2,607.14
E     $1,023.81
L    $27,591.97
S         $0.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_3264\1363743977.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'segmento
A       $26,309.77
B    $1,390,000.00
C       $28,273.97
D       $27,216.67
E   

In [42]:
import plotly.graph_objects as go
import os
import plotly.io as pio

# Crear la tabla con Plotly sin estilos personalizados
fig = go.Figure(data=[go.Table(
    header=dict(values=['Segmento','Mínimo', 'Máximo', 'Promedio', 'Mediana'],
                align='left'),
    cells=dict(values=[grupo_filtrado.index,  # Índices de categorías
                       grupo_filtrado['Mínimo'],  # Valores mínimos
                       grupo_filtrado['Máximo'],
                       grupo_filtrado['Promedio'],
                       grupo_filtrado['Mediana']],  # Valores máximos
               align='left'))
])
fig.show()

In [43]:
import plotly.io as pio
# Asegúrate de que la carpeta 'assets/tablas' exista
if not os.path.exists('assets/tablas'):
    os.makedirs('assets/tablas')

# Guardar cada fila en un archivo HTML separado
for index, row in grupo_filtrado.iterrows():
    # Crear la tabla para cada fila
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Segmento', 'Mínimo', 'Máximo', 'Promedio', 'Mediana'],
                    align='left'),
        cells=dict(values=[[row.name],  # Segmento actual usando el índice
                           [row['Mínimo']],  # Valor mínimo actual
                           [row['Máximo']],  # Valor máximo actual
                           [row['Promedio']],  # Promedio actual
                           [row['Mediana']]],  # Mediana actual
                       align='left'))
    ])

    # Nombre del archivo HTML basado en el nombre del índice (segmento)
    nombre_archivo = f"assets/tablas/precio_m2_construido_{row.name}.html"
    # Guardar la tabla como archivo HTML
    pio.write_html(fig, nombre_archivo)